DMD FUNCTIONS - FAT matrix X - for ONLINE DMD comparison - Zhang, Rowley,...

In [1]:
import numpy as np
import scipy.io
import matplotlib.pyplot as plt
from scipy import linalg

In [18]:
def GEDMD(X, Y, tol=1e-12, type: str=None, k=-1, weights=None): 
    """
    type = "exact" if you want exact version, or None (/leave empty) if you don't want exact version. 
    Anything else set as type will do the non-exact version.
    """
    m=X.shape[1]; N=X.shape[0]
    D = np.linalg.norm(X, axis=0)
    X = X/D; Y=Y/D
    if(weights is not None):
        X = X*weights; Y=Y*weights
    U, Sigma, V = np.linalg.svd(X,full_matrices=False) 
    V=np.conjugate(V.T)
    if(k==-1):
        k=m
        for i in range(1,min(N,m)):
            if(Sigma[i]<=Sigma[0]*tol):
                k=i
                break
    U = U[:, :k]; V=V[:, :k]; Sigma = np.array(Sigma[:k])
    B_k = Y @ (V/Sigma)
    S_k = np.conjugate(U.T)@B_k
    Lambda, W = np.linalg.eig(S_k)
    Z = U@W
    if(type=="exact"):
        Z = B_k@W
    #r = np.linalg.norm(B_k@W - Z*Lambda, axis=0)
    #return Z, Lambda, r
    return Z, Lambda

In [ ]:
def fGEDMDQ_multiple_trajectories(X, Y, tol=1e-12, type: str=None):
    m = X.shape[0]
    M = np.bmat([X.T, Y.T])
    Q, L = np.linalg.qr(M)
    L = L.T
    #Z, Lambda, r = GEDMD(L[:m,:m], L[m:,:m], tol, type)
    Z, Lambda= GEDMD(L[:m,:m], L[m:,:m], tol, type)
    Z = Z@Q[:,:m].T #provjeri - vjerojatno Q1
    #return Z, Lambda, r, Q, L 
    return Z, Lambda, Q, L

In [120]:
def fDMD_added_snap(v, Q, L, tol=1e-12, type: str=None, k=-1, weights=None, ngram=5):
    #v tipa 1 x 2m => prvih m je zadnji element u Y, drugih m je novi element
    # ngram - treba li tu reortogonalizacija?
    m=Q.shape[1]//2; n=Q.shape[0]
    print(m)
    R = np.bmat([[L.T],[v.T]])
    U, T = np.linalg.qr(R)
    T=T.T #novi L - sad je T donjetrokutasto
    Q = np.bmat([[Q@U[:2*m,:]],[U[-1,:]]])

    #novi Z i Lambda i r
    Z, Lambda= GEDMD(T[:m,:m], T[m:,:m], tol, type)
    Z = Z@(Q[:,:m].T)
    #return Z, Lambda, r, Q, T
    return Z, Lambda, Q, T



In [78]:
### Ovo bi trebalo biti isto?

def DMD_alpha_for_reconstruction(X, Z, indices, L, weights=None):
    """X = snapshotovi - prvih m (bez m+1-vog) - dakle X, a ne S
    Z = modes,
    indices = indices which we want to work with - from 1/r graph
    weights = np.array tip ili lista"""
    #treba li se formirati Vandermondeova matrica?
    m=X.shape[1]; l=indices.shape[0]
    Z = Z[:,indices]
    Q, R = np.linalg.qr(Z) # Q je tipa duljina_snapshota(N) x l, R je tipa lxl
    if(weights is None):
        weights=np.ones((m)).reshape(-1)
    weights=np.array(weights)
    pom=np.vander(L[indices], m, increasing=True)*weights
    alpha= np.multiply(np.conj(R.T)@R, np.conj(pom @ np.conj(pom.T)))
    G = (np.conj(Q.T) @ X)[:l, :]  
    alpha = scipy.linalg.solve(alpha, np.multiply(np.conj(pom),(np.conj(R.T)@G))@np.ones((m, 1)), assume_a='pos')
    #alpha = scipy.linalg.solve(alpha, np.multiply(np.conj(pom),(np.conj(R.T)@G*weights))@np.ones((m, 1)), assume_a='pos')
    return Z, L[indices], alpha.reshape(-1)

def DMD_reconstruction(X, Z, indices, L, times, weights=None, real=True): #mozda da prima vektor napraviti..
    """X = snapshotovi - prvih m (bez m+1-vog) - dakle X, a ne S,
    Z = dmd modes (returned from some version of DMD, ex. GEDMDQ)
    L = dmd eigs (returned from some version of DMD, ex. GEDMDQ)
    time = integer, which datasnapshot you want to reconstruct/predict"""
    Z_l, L, alpha = DMD_alpha_for_reconstruction(X, Z, indices, L, weights)
    num=np.asarray(times).shape[0]
    recs = np.empty((Z_l.shape[0], num), dtype='complex_')
    for i in range(num):
        recs[:,i] = Z_l@(L**(times[i])*alpha) #mislim da je times[i] jer ovdje krecemo od 0, za razliku od matlaba gdje krecemo od 1
    if real:
        return np.real(recs) #ako sve realno, ovo ce biti realno za svaki i, samo ce zapis biti u obliku kompleksnog; zato saljemo np.real(recs)!
    else:
        return recs


JEDNOSTAVNA PROVJERA

In [144]:
A=np.random.randint(0,5,(4,4)); A=A/np.linalg.norm(A)
x=np.ones((4,1))
y=A@x
Xevi = np.bmat([[np.copy(x)],[np.copy(y)]])
x=y
for i in range(30):
    y=A@x
    Xevi = np.bmat([Xevi, np.bmat([[np.copy(x)],[np.copy(y)]])])
    x=y

np.set_printoptions(2)
Xevi

matrix([[1.00e+00, 1.12e+00, 9.06e-01, 7.11e-01, 5.60e-01, 4.41e-01,
         3.48e-01, 2.74e-01, 2.16e-01, 1.70e-01, 1.34e-01, 1.06e-01,
         8.35e-02, 6.58e-02, 5.19e-02, 4.09e-02, 3.22e-02, 2.54e-02,
         2.00e-02, 1.58e-02, 1.25e-02, 9.82e-03, 7.74e-03, 6.10e-03,
         4.81e-03, 3.79e-03, 2.99e-03, 2.36e-03, 1.86e-03, 1.46e-03,
         1.15e-03],
        [1.00e+00, 6.12e-01, 3.85e-01, 3.02e-01, 2.39e-01, 1.88e-01,
         1.49e-01, 1.17e-01, 9.23e-02, 7.28e-02, 5.74e-02, 4.52e-02,
         3.56e-02, 2.81e-02, 2.21e-02, 1.75e-02, 1.38e-02, 1.09e-02,
         8.55e-03, 6.74e-03, 5.32e-03, 4.19e-03, 3.30e-03, 2.60e-03,
         2.05e-03, 1.62e-03, 1.28e-03, 1.01e-03, 7.93e-04, 6.25e-04,
         4.93e-04],
        [1.00e+00, 4.08e-01, 3.54e-01, 2.83e-01, 2.23e-01, 1.76e-01,
         1.38e-01, 1.09e-01, 8.61e-02, 6.78e-02, 5.35e-02, 4.22e-02,
         3.32e-02, 2.62e-02, 2.07e-02, 1.63e-02, 1.28e-02, 1.01e-02,
         7.98e-03, 6.29e-03, 4.96e-03, 3.91e-03, 3.08e-03, 2.43

In [145]:
np.linalg.eig(A)

(array([0.79+0.j  , 0.18+0.j  , 0.02+0.07j, 0.02-0.07j]),
 array([[ 0.59+0.j  , -0.78+0.j  ,  0.2 +0.18j,  0.2 -0.18j],
        [ 0.25+0.j  ,  0.61+0.j  , -0.72+0.j  , -0.72-0.j  ],
        [ 0.23+0.j  , -0.08+0.j  ,  0.26-0.24j,  0.26+0.24j],
        [ 0.73+0.j  ,  0.11+0.j  ,  0.49+0.19j,  0.49-0.19j]]))

In [146]:
GEDMD(Xevi[:4,:20], Xevi[4:,:20]) #iste sv. vrijednosti

(matrix([[ 0.59+0.j  ,  0.16+0.22j,  0.16-0.22j, -0.78+0.j  ],
         [ 0.25+0.j  , -0.7 -0.17j, -0.7 +0.17j,  0.61+0.j  ],
         [ 0.23+0.j  ,  0.31-0.17j,  0.31+0.17j, -0.08+0.j  ],
         [ 0.73+0.j  ,  0.43+0.31j,  0.43-0.31j,  0.11+0.j  ]]),
 array([0.79+0.j  , 0.02+0.07j, 0.02-0.07j, 0.18+0.j  ]))

In [147]:
Z, Lambda, Q, L = fGEDMDQ_multiple_trajectories(Xevi[:4,:20], Xevi[4:,:20])
Lambda #iste sv. vrij.

array([0.79+0.j  , 0.02+0.07j, 0.02-0.07j, 0.18+0.j  ])

In [152]:
Z, Lambda, Q, L = fDMD_added_snap(Xevi[:,20], Q, L)

4


In [153]:
m=4
indices = np.array([i for i in range(4)]); times=[i for i in range(31)]
reconstructed=DMD_reconstruction(Xevi[:m,:], Z, indices, Lambda, times)

In [154]:
reconstructed-Xevi[:m,:]

matrix([[ 2.08e-01, -4.29e-01, -4.54e-01, -3.72e-01, -2.96e-01,
         -2.34e-01, -1.84e-01, -1.45e-01, -1.15e-01, -9.03e-02,
         -7.12e-02, -5.61e-02, -4.42e-02, -3.49e-02, -2.75e-02,
         -2.17e-02, -1.71e-02, -1.35e-02, -1.06e-02, -8.37e-03,
         -6.60e-03, -5.20e-03, -4.10e-03, -3.23e-03, -2.55e-03,
         -2.01e-03, -1.58e-03, -1.25e-03, -9.84e-04, -7.76e-04,
         -6.11e-04],
        [-4.39e-02, -9.10e-01, -7.32e-01, -5.93e-01, -4.72e-01,
         -3.73e-01, -2.94e-01, -2.32e-01, -1.83e-01, -1.44e-01,
         -1.13e-01, -8.95e-02, -7.05e-02, -5.56e-02, -4.38e-02,
         -3.45e-02, -2.72e-02, -2.15e-02, -1.69e-02, -1.33e-02,
         -1.05e-02, -8.29e-03, -6.54e-03, -5.15e-03, -4.06e-03,
         -3.20e-03, -2.52e-03, -1.99e-03, -1.57e-03, -1.24e-03,
         -9.75e-04],
        [-7.07e-01, -9.51e-02, -9.36e-02, -7.35e-02, -5.71e-02,
         -4.49e-02, -3.53e-02, -2.79e-02, -2.20e-02, -1.73e-02,
         -1.36e-02, -1.08e-02, -8.48e-03, -6.69e-03, -5.27e-03